In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model
from keras.losses import mean_squared_error # (y_true, y_pred)
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/augument_data/training_images'
npy_img_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)\\augument_data\\training_images_batches'
architecture_plot_save_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'

img_width, img_height = 299., 299.
rx = 2.1404
ry = 1.61

In [3]:
##### img_width, img_height = 640, 480
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
# train_data_dir = "X:/train-scene classification/data/train/"
# validation_data_dir = "X:/train-scene classification/data/valid/"
# test_data_dir = "X:/train-scene classification/data/test/"
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_cust_loss_chkpt.h5" # Change for every new training session 

In [4]:
# base_model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3), classes = 4)
# base_model = InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (int(img_height), int(img_width), 3), classes = 4)

In [5]:
# base_model.save('inceptionresnetv2.h5')
base_model = load_model(os.path.join(base_dir,'base_models', 'inceptionresnetv2.h5'))

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
# plot_model(base_model, to_file= os.path.join(architecture_plot_save_dir, 'inceptiontresnetv2.png'))

In [7]:
for layer in base_model.layers[:-3]:
    layer.trainable = False
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 149, 149, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 149, 149, 32) 0           batch_normalization_204[0][0]    
__________________________________________________________________________________________________
conv2d_205

In [8]:
# for layer in base_model.layers:
#     layer.trainable = False
x = base_model.output
x = Flatten()(x)
# x = Dense(2048, activation="relu")(x)
# x = Dropout(0.7)(x)
# x = Dense(32, activation="relu")(x)
# x = Dropout(0.2)(x)
# x = Dense(16, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(6, activation="softmax")(x)
predictions = Dense(4, activation="relu")(x)

In [9]:
model = Model(input = base_model.input, output = predictions)
# for layer in model.layers[:-3]:
#     layer.trainable = False
model.summary()

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 149, 149, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 149, 149, 32) 0           batch_normalization_204[0][0]    
__________________________________________________________________________________________________
conv2d_205

conv2d_380 (Conv2D)             (None, 8, 8, 192)    399360      block8_3_ac[0][0]                
__________________________________________________________________________________________________
batch_normalization_380 (BatchN (None, 8, 8, 192)    576         conv2d_380[0][0]                 
__________________________________________________________________________________________________
activation_380 (Activation)     (None, 8, 8, 192)    0           batch_normalization_380[0][0]    
__________________________________________________________________________________________________
conv2d_381 (Conv2D)             (None, 8, 8, 224)    129024      activation_380[0][0]             
__________________________________________________________________________________________________
batch_normalization_381 (BatchN (None, 8, 8, 224)    672         conv2d_381[0][0]                 
__________________________________________________________________________________________________
activation

In [10]:
def mIoU(labels,predictions):
#     print(labels.shape)
#     print(predictions.shape)
#     print(labels[0])
#     print(predictions[0])
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [11]:
def custom_composite_loss(y_true, y_pred):
    iou = mIoU(y_true, y_pred)
    return tf.cond(tf.is_nan(iou),
                   true_fn=lambda: mean_squared_error(y_true, y_pred),
                   false_fn=lambda: mean_squared_error(y_true, y_pred) + ((1/(iou+1e-8) - 1)))

In [12]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

Tensor("Mean:0", shape=(), dtype=float64)
0.5714285714285714


#### Load best model weights from here, if required.

In [13]:
# model.load_weights(chkpt_dir)

In [14]:
model.compile(loss = custom_composite_loss, optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=['mse', mIoU])

In [15]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [16]:
train_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_train_data.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'augument_data', 'augmented_val_data.csv')))

In [17]:
l = []
for im in val_df.image_name:
    img = (cv2.imread(os.path.join(train_path, im))) ### INTRIDUCING NORMALIZATION.
    img = cv2.resize(img, (int(img_width), int(img_height)))
    img = (img / 255.).astype(np.float32)
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
print("x_val done.")
y_val = val_df.iloc[:,1:].values.astype(np.float32)
y_val[:, 0] = y_val[:, 0] # / img_width
y_val[:, 1] = y_val[:, 1] # / img_width
y_val[:, 2] = y_val[:, 2] # / img_height
y_val[:, 3] = y_val[:, 3] # / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)


x_val done.
[ 78. 578.  78. 317.]
float32
(2000, 299, 299, 3)


In [18]:
print(val_df.image_name[0])
# model.load_weights(chkpt_dir)

[h_v]1475157534672DSC07249.png


In [19]:
ram_batch_size = 1000
total_number_images = 68000
print(total_number_images // ram_batch_size)

68


In [20]:
def show_img_with_rect(img, lab):
    x1 = int(lab[0]/2.1404)
    x2 = int(lab[1]/2.1404)
    y1 = int(lab[2]/1.61)
    y2 = int(lab[3]/1.61) #
    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [21]:
# show_img_with_rect(x_val[654], y_val[654])

In [ ]:
ctr = 0

for epoch in range(epochs):
    print(epoch)
    print("--------------------------------------------------------------")
    for i in range(total_number_images // ram_batch_size):
        print('batch {}/67 (epoch {}):'.format(i, epoch))
        l = []
#         for im in train_df.image_name[i * ram_batch_size: (i + 1) * ram_batch_size]:
#             img = (cv2.imread(os.path.join(train_path, im)) / 255.).astype(np.float32) ### INTRIDUCING NORMALIZATION.
#             l.append(np.expand_dims(img, axis = 0))
        x_train = np.load(os.path.join(npy_img_dir, str(i) + '.npy'))
        y_train = train_df[i * ram_batch_size: (i + 1) * ram_batch_size].iloc[:,1:].values
#         y_train[:, 0] = y_train[:, 0] # / img_width
#         y_train[:, 1] = y_train[:, 1] # / img_width
#         y_train[:, 2] = y_train[:, 2] # / img_height
#         y_train[:, 3] = y_train[:, 3] # / img_height
#         plt.figure(i)
#         show_img_with_rect(x_train[0], y_train[0])
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)
#         if i == 0:
#             pred = model.predict(np.expand_dims(x_train[0], axis=0))
#             X1 = int(pred[0][0]/rx)
#             X2 = int(pred[0][1]/rx)
#             Y1 = int(pred[0][2]/ry)
#             Y2 = int(pred[0][3]/ry)
#             cv2.rectangle(x_train[0],(X1,Y1),(X2,Y2),(0,255,0),3)
#             plt.figure(ctr)
#             ctr += 1
#             plt.imshow(cv2.cvtColor(x_train[0], cv2.COLOR_BGR2RGB))

0
--------------------------------------------------------------
batch 0/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 34s 34ms/step - loss: 12839302.4196 - mean_squared_error: 38779.9041 - mIoU: 0.1946 - val_loss: 24262.5346 - val_mean_squared_error: 24260.3841 - val_mIoU: 0.3205

Epoch 00001: val_loss improved from inf to 24262.53459, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_cust_loss_chkpt.h5
batch 1/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 7710.0427 - mean_squared_error: 7708.9637 - mIoU: 0.4851 - val_loss: 27366.8219 - val_mean_squared_error: 27362.1363 - val_mIoU: 0.1793

Epoch 00001: val_loss did not improve from 24262.53459
batch 2/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 2

batch 19/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 988.2119 - mean_squared_error: 987.9173 - mIoU: 0.7732 - val_loss: 2352.2235 - val_mean_squared_error: 2351.6621 - val_mIoU: 0.6415

Epoch 00001: val_loss did not improve from 1399.98339
batch 20/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 884.4218 - mean_squared_error: 884.1341 - mIoU: 0.7770 - val_loss: 1953.1872 - val_mean_squared_error: 1952.6934 - val_mIoU: 0.6703

Epoch 00001: val_loss did not improve from 1399.98339
batch 21/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 900.0301 - mean_squared_error: 899.7357 - mIoU: 0.7731 - val_loss: 1460.0818 - val_mean_squared_error: 1459.6866 - val_mIoU: 0.7176

Epoch 00001: val_loss did not improve from 1399.9

1000/1000 [==============================] - 27s 27ms/step - loss: 689.9189 - mean_squared_error: 689.6807 - mIoU: 0.8084 - val_loss: 1208.8998 - val_mean_squared_error: 1208.5493 - val_mIoU: 0.7410

Epoch 00001: val_loss did not improve from 974.65908
batch 65/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 604.8957 - mean_squared_error: 604.6743 - mIoU: 0.8191 - val_loss: 1025.8051 - val_mean_squared_error: 1025.4978 - val_mIoU: 0.7655

Epoch 00001: val_loss did not improve from 974.65908
batch 66/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 598.5711 - mean_squared_error: 598.3441 - mIoU: 0.8154 - val_loss: 1417.8983 - val_mean_squared_error: 1417.4930 - val_mIoU: 0.7122

Epoch 00001: val_loss did not improve from 974.65908
batch 67/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/

1000/1000 [==============================] - 27s 27ms/step - loss: 441.9239 - mean_squared_error: 441.7198 - mIoU: 0.8308 - val_loss: 1375.7543 - val_mean_squared_error: 1375.3373 - val_mIoU: 0.7064

Epoch 00001: val_loss did not improve from 956.95072
batch 21/67 (epoch 1):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 469.7914 - mean_squared_error: 469.5852 - mIoU: 0.8293 - val_loss: 1099.4308 - val_mean_squared_error: 1099.0923 - val_mIoU: 0.7477

Epoch 00001: val_loss did not improve from 956.95072
batch 22/67 (epoch 1):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 486.5291 - mean_squared_error: 486.3221 - mIoU: 0.8289 - val_loss: 1026.1156 - val_mean_squared_error: 1025.8079 - val_mIoU: 0.7653

Epoch 00001: val_loss did not improve from 956.95072
batch 23/67 (epoch 1):
Train on 1000 samples, validate on 2000 samples
Epoch 1/

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 406.4841 - mean_squared_error: 406.2899 - mIoU: 0.8379 - val_loss: 1154.5988 - val_mean_squared_error: 1154.2211 - val_mIoU: 0.7265

Epoch 00001: val_loss did not improve from 886.21985
batch 1/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 416.3160 - mean_squared_error: 416.1170 - mIoU: 0.8345 - val_loss: 1250.3202 - val_mean_squared_error: 1249.9323 - val_mIoU: 0.7212

Epoch 00001: val_loss did not improve from 886.21985
batch 2/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 352.2458 - mean_squared_error: 352.0679 - mIoU: 0.8492 - val_loss: 975.2128 - val_mean_squared_error: 974.8972 - val_mIoU: 0.7606

Epoch 00001: val_loss did not improve from 886.21985
batch 3/67 (epoch 2):
Tra


Epoch 00001: val_loss did not improve from 886.21985
batch 25/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 335.8390 - mean_squared_error: 335.6592 - mIoU: 0.8478 - val_loss: 1572.0584 - val_mean_squared_error: 1571.5890 - val_mIoU: 0.6815

Epoch 00001: val_loss did not improve from 886.21985
batch 26/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 360.6608 - mean_squared_error: 360.4752 - mIoU: 0.8439 - val_loss: 1255.5913 - val_mean_squared_error: 1255.1810 - val_mIoU: 0.7098

Epoch 00001: val_loss did not improve from 886.21985
batch 27/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 327.0476 - mean_squared_error: 326.8665 - mIoU: 0.8469 - val_loss: 1210.1492 - val_mean_squared_error: 1209.7581 - val_mIoU: 0.719

batch 49/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 305.3896 - mean_squared_error: 305.2154 - mIoU: 0.8519 - val_loss: 880.4198 - val_mean_squared_error: 880.1343 - val_mIoU: 0.7784

Epoch 00001: val_loss did not improve from 852.67306
batch 50/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 308.2066 - mean_squared_error: 308.0292 - mIoU: 0.8497 - val_loss: 1258.3362 - val_mean_squared_error: 1257.9329 - val_mIoU: 0.7134

Epoch 00001: val_loss did not improve from 852.67306
batch 51/67 (epoch 2):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 316.4459 - mean_squared_error: 316.2630 - mIoU: 0.8457 - val_loss: 1361.8923 - val_mean_squared_error: 1361.4527 - val_mIoU: 0.6956

Epoch 00001: val_loss did not improve from 852.67306



Epoch 00001: val_loss did not improve from 852.67306
batch 30/67 (epoch 3):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 271.4126 - mean_squared_error: 271.2480 - mIoU: 0.8591 - val_loss: 928.8183 - val_mean_squared_error: 928.5089 - val_mIoU: 0.7642

Epoch 00001: val_loss did not improve from 852.67306
batch 31/67 (epoch 3):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 250.9901 - mean_squared_error: 250.8282 - mIoU: 0.8609 - val_loss: 992.8330 - val_mean_squared_error: 992.5157 - val_mIoU: 0.7596

Epoch 00001: val_loss did not improve from 852.67306
batch 32/67 (epoch 3):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 290.0732 - mean_squared_error: 289.9125 - mIoU: 0.8619 - val_loss: 1268.9081 - val_mean_squared_error: 1268.5105 - val_mIoU: 0.7163

E


Epoch 00001: val_loss did not improve from 852.67306
batch 11/67 (epoch 4):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 213.5172 - mean_squared_error: 213.3694 - mIoU: 0.8714 - val_loss: 955.9594 - val_mean_squared_error: 955.6292 - val_mIoU: 0.7524

Epoch 00001: val_loss did not improve from 852.67306
batch 12/67 (epoch 4):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 214.2887 - mean_squared_error: 214.1410 - mIoU: 0.8715 - val_loss: 1269.3533 - val_mean_squared_error: 1268.9491 - val_mIoU: 0.7130

Epoch 00001: val_loss did not improve from 852.67306
batch 13/67 (epoch 4):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 235.7691 - mean_squared_error: 235.6081 - mIoU: 0.8616 - val_loss: 1044.9823 - val_mean_squared_error: 1044.6466 - val_mIoU: 0.7492


In [ ]:
model.save('inceptionresnetv2_cust_loss.h5')

In [ ]:
for i in range(0, 2000, 100):
    pred = model.predict(np.expand_dims(x_val[i], axis=0))
    a1 = int(pred[0][0]/rx)
    a2 = int(pred[0][1]/rx)
    b1 = int(pred[0][2]/ry)
    b2 = int(pred[0][3]/ry)
    test_img = x_val[i]
    print(test_img.shape)
    plt.figure(i)
    cv2.rectangle(test_img,(a1,b1),(a2,b2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

In [ ]:
test_img = x_val[7]
test_img = np.expand_dims(test_img, axis=0)
pred = model.predict(test_img)
print(pred.shape)
a1 = int(pred[0][0]/rx)
a2 = int(pred[0][1]/rx)
b1 = int(pred[0][2]/ry)
b2 = int(pred[0][3]/ry)
test_img = test_img[0]
cv2.rectangle(test_img,(a1,b1),(a2,b2),(0,255,0),3)
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

In [ ]:
# model.save('inceptionresnetv2_cust_loss.h5')

In [ ]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

In [ ]:
test_df = pd.read_csv(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data', 'test.csv'))

In [ ]:
print(type(test_df))

In [ ]:
for i, im in enumerate(test_df.image_name):
    img = cv2.imread(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data/train', im))